In [101]:
import pandas as pd
from collections import Counter
import re
import math
df = pd.read_csv("Data/2019_TFIDF_DBSCAN_CV_eps8_mp20.csv", encoding='utf-8-sig')

In [2]:
df = df.sort_values(by='labels', ascending=True)
df.tail()

,Unnamed: 0,N_list,category,date,headline,img,press,text,url,topics,labels,cnt
3091,124855,"[['서울시', '의회', '우리공화당', '광화문광장', '천막', '서울시', ...","사회,지역",2019-06-28,서울시의회 “우리공화당 불법 천막 규탄…엄중히 대처 할 것” 촉구 결의안 가결,['//img.segye.com/content/image/2019/06/28/201...,세계일보,서울시의회는 우리공화당(구 대한애국당)의 광화문광장 천막에 대해 서울시가 엄정한 법...,http://www.segye.com/content/html/2019/06/28/2...,"우리공화당, 천막, 철거, 광화문광장, 집행, 설치, 불법, 시의회, 공권력, 결의안",59,61
3090,121811,"[['대한애국당', '기습', '광화문광장', '천막', '동', '설치'], ['...","사회,국제",2019-06-25,"공화당, 천막 철거 3시간 만에 재설치…서울시 ""단호 대응""",['https://pds.joins.com/news/component/htmlpho...,중앙일보,대한애국당(현 우리공화당)이 기습적으로 광화문광장에 천막 3동을 다시 설치했다. 서...,https://news.joins.com/article/olink/23098468,"천막, 철거, 서울시, 공화당, 설치, 용역, 우리공화당, 계획, 대한애국당, 광화문광장",59,61
3089,125620,"[['우리공화당', '행정', '집행', '서울시', '근본', '대책', '경찰'...","사회,국제",2019-06-30,'광화문 천막 사태' 재발 우려…트럼프 떠난 뒤 다시 칠 가능성,['//img.segye.com/content/image/2019/06/30/201...,세계일보,"우리공화당, 세 번째 설치땐 행정대집행 불가피서울시 ""근본대책 절실""경찰, 시설물보...",http://www.segye.com/content/html/2019/06/30/2...,"천막, 우리공화당, 광화문광장, 서울시, 행정, 집회, 집행, 철거, 경찰, 광장",59,61
3087,90091,"[['강제', '철거', '대한애국당', '기습', '설치', '광화문광장', '농...","사회,정치",2019-05-13,서울시 “애국당 천막 세월호와 달라”… 강제철거는 14일부터 가능,['//img.segye.com/content/image/2019/05/13/201...,세계일보,강제철거는 14일부터 가능대한애국당이 기습적으로 설치한 광화문광장 농성 천막에 대해...,http://www.segye.com/content/html/2019/05/13/2...,"철거, 천막, 서울시, 자진, 설치, 대한애국당, 불법, 강제, 상금, 광화문광장",59,61
3123,124795,"[['우리공화당', '광화문광장', '불법', '설치', '천막', '자진', '철...","국제,정치",2019-06-28,"“트럼프를 질서있게 환영하기 위해” 공화당, 광화문광장 천막 철거",['http://image.kmib.co.kr/online_image/2019/06...,국민일보,우리공화당이 광화문광장에 불법으로 설치한 천막을 28일 자진 철거했다. 지난달 10...,http://news.kmib.co.kr/article/view.asp?arcid=...,"천막, 우리공화당, 철거, 광화문광장, 자진, 서울시, 설치, 조원진, 환영, 트럼프",59,61


In [3]:
def npmi(a, b, n, z):
    x = math.log10((n*z)/(a*b))
    y = math.log10(n/z)
    return x / y

In [4]:
# 뉴스 기사별 PMI 체크
merge_tokens = []
docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []
    merge_token2 = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        doc.append(arr)
        context += arr

    #2 PMI 적용
    for line in doc:
        com += list(zip(line, line[1:]))
    n = len(context)
    cnt = Counter(context)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        if -1 < pmi < 1:
            nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+"_"+key[1] for key, item in result if item >= 0.91]
    if len(merge_token) > 0:
        merge_tokens+=merge_token
    # 토큰을 합치는 내용
    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = '_'.join(content[i:i+2])
                if cmp_str == token:
                    del content[i:i+2]
                    content.insert(i, token)
    docs.append(doc)
z = Counter(merge_tokens).most_common()
print(z)

'거부_권리', 1), ('산부인과_의사', 1), ('형법_조항', 1), ('위헌_의견', 1), ('성장_단계', 1), ('임부_자기', 1), ('일선_법원', 1), ('논의_염두', 1), ('여성_자기', 1), ('유감_표', 1), ('서울_교구', 1), ('규정_폐지', 1), ('인_시위', 1), ('이번_헌재', 1), ('임신_중절', 1), ('중절_허용', 1), ('일률_금지', 1), ('유감_표시', 1), ('권리_존중', 1), ('후속_조치', 1), ('단순_위헌', 1), ('목적_달성', 1), ('해당_법률', 1), ('태아_모체', 1), ('각계_의견', 1), ('의견_경청', 1), ('유지_출산', 1), ('보호_공익', 1), ('처벌_조항', 1), ('성장_상태', 1), ('염_추기경', 1), ('존엄_무시', 1), ('임신부_기준', 1), ('국회_보완', 1), ('공개_변론', 1), ('소원_청구', 1), ('죄_처벌', 1), ('임산부_자기', 1), ('보건_법', 1), ('공동_운영', 1), ('생산_논의', 1), ('헌재_다음', 1), ('위헌_여부', 1), ('생산_침해', 1), ('건강_생명', 1), ('형사_처벌', 1), ('헌법_재판소', 1), ('역사_마침표', 1), ('후속_입법', 1), ('우선_여론', 1), ('한도_제시', 1), ('산모_건강', 1), ('신중_입장', 1), ('제한_사익', 1), ('남성_복수', 1), ('회견_예정', 1), ('최선_의료', 1), ('구체_허용', 1), ('산부인과_의사회', 1), ('음성_부작용', 1), ('상태_양육', 1), ('대한_산부인과', 1), ('과도_침해', 1), ('배당_소득', 1), ('배제_관측', 1), ('경상_수지', 1), ('외부_요인', 1), ('감소_작년', 1), ('미중_무역', 1), ('화물_터미널', 1), ('경기_회복', 1)

In [27]:
df['N_list'] = docs
df['tokens'] = merge_tokens
df.to_csv("./Data/2019_DBSCAN.csv", encoding="utf-8-sig", index=False)

ValueError: Length of values (8258) does not match length of index (4397)

In [5]:
# 클러스터별 pmi 체크
cluster = {}
for i in range(df['labels'].max()+1):
    print(i,": ================")
    word_list = []
    com=[]
    for article in df[df['labels'] == i]['N_list']:
        # str -> list
        for line in article.split('], ['):
            arr = re.sub("[\[,'\]]", '', line).split(' ')
            word_list += arr
            com += list(zip(arr, arr[1:]))
    print(len(word_list))
    print(len(com))
    n = len(word_list)
    cnt = Counter(word_list)
    nmi = {}
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = [key[0]+"_"+key[1] for key, item in result if item >= 0.9]
    # merge_token = [(key, item) for key, item in result if item >= 0.9]
    print(merge_token)
    cluster[i] = merge_token

회동_타진', '연철_정유경', '정국_강타', '동원_국정', '국정_농', '개인_일탈', '해임_파면', '온정주의_신속', '태극_국무', '칼바람_관측', '개인_학연', '자료_토대', '전면_부인', '압박_계속', '되풀이_당국', '방해_대처', '차_북미', '베트남_하노이', '시설_폐쇄', '시설_영변', '강선_나머지', '나머지_곳', '여권_지원', '여직원_사생활', '교육부_통보', '자행_세상', '세상_적법', '적법_세상', '기자회견_서', '메시지_발신', '라디오_관용', '시사자키_인터뷰', '보존_이것', '대북_메시지', '누_통화록', '싸움_발단', '트랙_독재자', '벼랑_끝', '끝_대치', '격돌_정국', '고발장_접수', '해이_총체', '물타기_성질', '국_오기', '의도_용납', '파리_특파원', '특파원_간담회', '본부_암호', '기만_민낯', '대책_마련', '권한_위임', '공세_동조', '냉정_이슈', '수색_촛불', '박지원_민주평화당', '의견_교환', '이의_제기', '한_미', '우선_가치', '의전_실무', '운운_어이']
52 : ================
4028
3678
['총무_기획', '김성호_원세훈', '원세훈_국정원장', '특수_활동비', '예산_전용', '국고_손실', '공소_시효', '서초_경찰서', '권한_행사', '소_환장', '능력_증명력', '가족_재산', '정부_시절', '거리_보좌', '의혹_김석', '사망_도피', '염_유섭', '특정_범죄', '범죄_가중', '가중_처벌', '처벌_법률', '고지_대상', '이팔성_우리금융지주', '이학수_삼성그룹', '투자금_회수', '보직_임명', '임명_대가', '그날_당사자', '연속_주말', '심신_쇠약', '쇠약_정신', '정신_체력', '체력_기력', '기력_회복', '국민일보_무단', '무단_전재', '전재_수집', '수집_배포', '배포_금지', '구금_헌법', '헌법_기본', '기본_침해', '성

In [6]:
# 뉴스 기사별 N_GRAM 체크
merge_tokens = []
docs = []
for article in df['N_list']:
    doc = []
    com = []
    context = []
    merge_token = []

    # str -> list
    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        doc.append(arr)
        com += list(zip(arr, arr[1:]))

    #2 N_GRAM 체크
    z = Counter(com).most_common()
    for item, value in z:
        if value > 3:
            merge_token.append(item[0]+'_'+item[1])
    if len(merge_token) > 0:
        merge_tokens += merge_token

    # 토큰을 합치는 내용
    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = '_'.join(content[i:i+2])
                if cmp_str == token:
                    del content[i:i+2]
                    content.insert(i, token)
    docs.append(doc)
z = Counter(merge_tokens).most_common()
print(z)

 2), ('대_국회', 2), ('황_의원', 2), ('개혁_법안', 2), ('당_원내대표', 2), ('오_의원', 2), ('회장_구속', 2), ('김성태_의원', 2), ('채용_건', 2), ('검사_결과', 2), ('국과수_검사', 2), ('박_측', 2), ('잠정_중단', 2), ('규모_시위', 2), ('무기_발사', 2), ('회계_사기', 2), ('안_부사장', 2), ('삼성전자_사업', 2), ('부산_버스', 2), ('조정_회의', 2), ('이날_오전', 2), ('공무원_시험', 2), ('동월_대비', 2), ('대비_포인트', 2), ('고용_포인트', 2), ('대_대', 2), ('포인트_상승', 2), ('천_감소', 2), ('대_천', 2), ('세_이상', 2), ('고용_질', 2), ('조_신임', 2), ('일방_제재', 2), ('와이즈_어니스트', 2), ('어니스트_호', 2), ('선박_압류', 2), ('북한_화물선', 2), ('미국_선박', 2), ('기밀_유출', 2), ('대통령_항소심', 2), ('증인_출석', 2), ('항소심_재판', 2), ('침입_죄', 2), ('긴급_체포', 2), ('피해_여성', 2), ('선체_인양', 2), ('침몰_사고', 2), ('남성_시신', 2), ('헝가리_당국', 2), ('수색_요원', 2), ('작업_진행', 2), ('송_대령', 2), ('강_하류', 2), ('하류_쪽', 2), ('수면_위', 2), ('진상_조사', 2), ('결과_발표', 2), ('조사_팀', 2), ('장자연_사건', 2), ('외부_위원', 2), ('검찰_과거사위', 2), ('김학의_사건', 2), ('정무_수석', 2), ('주권자_국민', 2), ('전북_교육감', 2), ('취소_동의', 2), ('상산고_자사', 2), ('취소_결정', 2), ('통합_전형', 2), ('변_상금', 2), ('대한애국당_천막', 2), ('용역_업체'

In [7]:
#클러스터별 N GRAM


for i in range(df['labels'].max()+1):
    docs = []
    merge_tokens = []
    print(i, "===================")
    for article in df[df['labels'] == i]['N_list']:
        doc = []
        com = []
        context = []
        merge_token = []

        # str -> list
        for line in article.split('], ['):
            arr = re.sub("[\[,'\]]", '', line).split(' ')
            doc.append(arr)
            com += list(zip(arr, arr[1:]))

        #2 N_GRAM 체크
        z = Counter(com).most_common()
        for item, value in z:
            if value > 3:
                merge_token.append(item[0]+'_'+item[1])
        if len(merge_token) > 0:
            merge_tokens += merge_token

        # 토큰을 합치는 내용
        for token in merge_token:
            for content in doc:
                for i in range(len(content)-1):
                    cmp_str = '_'.join(content[i:i+2])
                    if cmp_str == token:
                        del content[i:i+2]
                        content.insert(i, token)
        docs.append(doc)
    z = Counter(merge_tokens).most_common()
    print(z)

프', 1), ('미국_대통령', 1), ('대통령_아베', 1), ('문_대통령', 1)]
21 ===================
[('탄력_근로', 42), ('단위_기간', 30), ('근로_단위', 17), ('기간_확대', 10), ('근로_확대', 8), ('임금_보전', 7), ('합의_안', 5), ('초과_탄력', 5), ('확대_적용', 3), ('노사_합의', 3), ('노동_제도', 3), ('제도_개선', 3), ('개선_위', 3), ('최저_임금', 2), ('근로_도입', 2), ('김_위원장', 2), ('운용_기간', 2), ('주_노동', 2), ('노동_개선', 2), ('서면_합의', 2), ('임금_결정', 1), ('결정_체계', 1), ('문_대통령', 1), ('법_개정', 1), ('근로_기간', 1), ('경제사회노동위원회_노동', 1), ('개선_위원회', 1), ('환경_노동', 1), ('노동_위원회', 1), ('근로_논의', 1), ('고용_노동', 1), ('경사노위_합의', 1), ('건강_보호', 1), ('보전_방안', 1), ('노동자_대표', 1), ('노_사', 1), ('입장_차', 1), ('경사노위_논의', 1)]
22 ===================
[('김_장관', 49), ('산하_기관', 45), ('기관_임원', 25), ('환경부_산하', 20), ('구속_영장', 12), ('인사_수석실', 7), ('환경부_블랙리스트', 6), ('장관_구속', 6), ('청와대_인사', 5), ('영장_기각', 5), ('영장_청구', 5), ('검찰_김', 4), ('상임_감사', 3), ('기관_인사', 2), ('직권_남용', 2), ('수석실_보고', 1), ('청와대_낙점', 1), ('부장_판사', 1), ('내정_인사', 1), ('청와대_내정', 1), ('정_국장', 1), ('박_차관', 1), ('공공_기관', 1), ('박_탈락', 1), ('기각_사유', 1

In [155]:
from gensim.models import KeyedVectors

loaded_model=KeyedVectors.load("word_vec2")
loaded_model.most_similar("")

[('경협', 0.43442708253860474),
 ('김영철', 0.40458330512046814),
 ('고대', 0.3864108920097351),
 ('매파', 0.3814399540424347),
 ('노골적', 0.38012757897377014),
 ('통일전선부', 0.37808918952941895),
 ('조미수뇌', 0.3741137385368347),
 ('이발', 0.37381061911582947),
 ('차 북미', 0.36483627557754517),
 ('대톨령', 0.3632027804851532)]

In [154]:
loaded_model=KeyedVectors.load("word_fast2")
loaded_model.most_similar("정은")

[('이정은', 0.8745384812355042),
 ('페이지', 0.8439659476280212),
 ('병기', 0.8291098475456238),
 ('정착', 0.819145917892456),
 ('방침', 0.7895593643188477),
 ('디', 0.7649025917053223),
 ('상임', 0.7409874200820923),
 ('김정은', 0.7392602562904358),
 ('국가안전보장회', 0.7311857342720032),
 ('파악', 0.6985346078872681)]

In [98]:
import pandas as pd
from gensim.models import word2vec
from gensim.models import Word2Vec


word_list = []

for i in range(df['labels'].max()+1):
    for article in df[df['labels'] == i]['N_list']:
        doc = []
        for line in article:
            doc += line
            word_list.append(doc)
    
print(len(word_list))
my_model = Word2Vec(word_list, window = 100, min_count = 1, sg = 1,workers=4,vector_size=200)
word_vectors = my_model.wv
word_vectors.save('word_vec2')

81673


In [8]:
import pandas as pd
from gensim.models import word2vec


word_list = []
for article in df[df['labels'] == 36]['N_list']: #번호에 해당 클러스터 번호입력
    doc = []
    for line in article:
        doc += line
        word_list.append(doc)

print(len(word_list))
my_model_fast = FastText(word_list, window = 100, min_count = 1, sg = 1,vector_size=100)

word_vectors_fast = my_model_fast.wv
word_vectors_fast.save('word_fast2')


C:\Users\zoo58\AppData\Local\Programs\Python\Python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
43752


NameError: name 'FastText' is not defined

In [11]:
df['N_list']

0       [['류허', '부총리', '면담', '부총리', '협상', '진전', '트럼프',...
719     [['도널드 트럼프', '미국', '대통령', '미', '무역', '협상', '직전...
720     [['미국', '중국 협상단', '류허', '부총리', '달', '이내', '합의'...
721     [['무역', '협상', '치열', '기싸움미', '무역', '협상', '치', '...
722     [['트럼프', '뒷짐', '도널드 트럼프', '미국', '대통령', '연합뉴스',...
                              ...                        
3091    [['서울시', '의회', '우리공화당', '광화문광장', '천막', '서울시', ...
3090    [['대한애국당', '기습', '광화문광장', '천막', '동', '설치'], ['...
3089    [['우리공화당', '행정', '집행', '서울시', '근본', '대책', '경찰'...
3087    [['강제', '철거', '대한애국당', '기습', '설치', '광화문광장', '농...
3123    [['우리공화당', '광화문광장', '불법', '설치', '천막', '자진', '철...
Name: N_list, Length: 4397, dtype: object

In [137]:
import pandas as pd

df = pd.read_csv("Data/2019_TFIDF_DBSCAN_CV_eps8_mp20.csv", encoding='utf-8-sig')

tokenlist={}

with open("tokenlist.txt",'r',encoding='utf-8') as f:
    for line in f:
        token,word=line.split(',')
        tokenlist[token]=word.strip()

front=[]
end=[]

for key in tokenlist.keys():
    first,second=key.split('_')
    front.append(first)
    end.append(second)

docs=[]

for article in df['N_list']:

    doc=[]

    for line in article.split('], ['):
        arr = re.sub("[\[,'\]]", '', line).split(' ')
        del_nums=[]
        new_arr=[]
        for i in range(len(arr)-1):
            if arr[i] in front:
                if arr[i+1]==end[front.index(arr[i])]:
                    #print(arr[i],arr[i+1])
                    del_nums.append(i)
                    del_nums.append(i+1)
                    #arr.append(arr[i]+" "+arr[i+1])
                    arr.append(tokenlist[arr[i]+"_"+arr[i+1]])

      
        for i in range(len(arr)):
            if(i not in list(set(del_nums))):
                new_arr.append(arr[i])   
        
        doc.append(new_arr)
        
        
    docs.append(doc)

#docs
print("Finished")
    
        # for key in tokenlist.keys():
        #     front,end=key.split('_')

        #     for i in range(len(arr)-1):
        #         if(arr[i]==front):
        #             if(arr[i+1]==end):
        #                 #print(arr[i],arr[i+1])
        #                 new_arr.append(arr[i]+arr[i+1])
        #                 i=i+2
        #         else:
        #             new_arr.append(arr[i])

       



Finished


In [138]:
df2=df
df2['N_list']=docs
df2.to_csv("./Data/2019_DBSCAN_new.csv", encoding="utf-8-sig", index=False)

In [58]:
import pandas as pd
from collections import Counter
import re

df = pd.read_csv("./Data/2019_DBSCAN_new.csv", encoding='utf-8-sig')

df_out=pd.DataFrame()

words=[]

for i in range(df['labels'].max()+1):
    word_list=[]
    #print(i,'-----------------')
    for article in df[df['labels'] == i]['N_list']:
        for line in article.split('], ['):
            arr = re.sub("[\[,'\]]", '', line).split(' ')
            word_list+=arr
    count=Counter(word_list).most_common(20)
    word = [key for key, item in count]
    
    words.append(word)

df_out['topics']=words

df_out.to_csv("./Data/2019_cluster_topics2.csv", encoding="utf-8-sig", index=False)

In [187]:
import pandas as pd

df = pd.read_csv("Data/헝가리 유람선 사고.csv", encoding='cp949')
df

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,1.101001e+06,20190628,한겨레,노지원,“아직 두명을 더 찾아야 한다” 다뉴브강 유람선 침몰 실종자 1명 신원 확인,국제>유럽_EU,국제>아시아,사회>여성,사고>교통사고>해상사고,NaN,NaN,NaN,"헝가리,부다페스트,체펠(Csepel),체펠섬,한국,허블레아니호,다뉴브강","외교부,정부","확인,다뉴브강,유람선,침몰,실종자,신원,닷새,헝가리,부다페스트,다뉴브강,인근,체펠섬...","실종자,헝가리,한국인,다뉴브강,부다페스트,신속대응팀,유람선,허블레아니호,노지원,사망...",닷새 전 헝가리 부다페스트 다뉴브강 인근 체펠섬 지역에서 발견된 여성의 시신이 60...,http://www.hani.co.kr/arti/society/society_gen...,NaN
1,1.100401e+06,20190628,동아일보,김소정,"외교부 “22일 헝가리 유람선 수습 시신, 60대 한국인 여성” 실종자 2명",사회>여성,국제>유럽_EU,국제>아시아,NaN,NaN,NaN,NaN,"헝가리,체펠섬,한국,다뉴브강","외교부,정부","외교,헝가리,유람선,수습,시신,여성,한국인,실종자,2명,22일,현지시간,헝가리,다뉴...","한국인,헝가리,생존자,사망자,실종자,탑승객,외교부,한국시간,체펠섬,한국,2명,다뉴브...",22일(현지시간) 헝가리 다뉴브강에서 수습된 시신이 60대 한국인 여성으로 확인됐다...,http://www.donga.com/news/article/all/20190628...,NaN
2,1.100611e+06,20190628,서울신문,오세진,‘헝가리 유람선 침몰’ 24번째 한국인 탑승객 사망자 신원 확인,사회>사회일반,국제>유럽_EU,국제>아시아,NaN,NaN,NaN,"최규식,에르빈,강형식","헝가리,부다페스트,주헝가리,체펠섬,한국,허블레아니호,다뉴브강","신속대,헝가리,허블레아니호 침몰,정부,바이킹,한국대사,외교부,신원,한국,검찰청,해외...","침몰,헝가리,유람선,24번,확인,한국인,탑승객,사망자,신원,실종자,2명,헝가리,한국...","한국인,헝가리,실종자,벨로비츠,시긴호,부다페스트,한국,허블레아니호,강형식,탑승객,승...",헝가리가 한국과 함께 허블레아니호 침몰 참사의 실종자 수색 작업을 계속하는 가운데 ...,http://www.seoul.co.kr/news/newsView.php?id=20...,NaN
3,1.100401e+06,20190628,동아일보,위은지,"헝가리 다뉴브강서 22일 수습한 시신, 60대 한국인 여성 확인",국제>유럽_EU,국제>아시아,사회>여성,NaN,NaN,NaN,NaN,"헝가리,부다페스트,체펠섬,한국,허블레아니호,다뉴브강","허블레아니호,다뉴브강,정부합동신속대응팀","헝가리,다뉴브강서,수습,시신,한국인,여성,확인,침몰사고,헝가리,부다페스트,유람선,허...","한국인,헝가리,허블레아니호,다뉴브강,유람선,dna,관광객,다뉴브강서,머그리트,체펠섬...",헝가리 부다페스트 유람선 허블레아니호 침몰사고와 관련해 22일 밤(현지 시간) 다뉴...,http://www.donga.com/news/article/all/20190628...,NaN
4,1.100901e+06,20190628,중앙일보,권혜림(kwon.hyerim@joongang.co.kr),"'다뉴브 참사' 22일 발견 시신, 한국인 탑승객으로 확인 남은 실종자 2명",국제>아시아,국제>유럽_EU,사회>사건_사고,NaN,NaN,NaN,NaN,"주헝가리,서한,헝가리,부다페스트,한국,다뉴브강","허블레아니호,정부합동신속대응팀,한국대사","다뉴브,참사,시신,발견,한국인,탑승객,확인,실종자,헝가리,다뉴브강,발생,사고,유람선...","헝가리,실종자,한국인,한국,검찰총장,벨로비츠,주헝가리,dna,유람선,삐테르,체펠,정...","헝가리 다뉴브강에서 발생한 유람선 침몰 사고와 관련, 지난 22일(현지시간) 수습된...",https://news.joins.com/article/olink/23101810,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,1.100701e+06,20190530,세계일보,NaN,"헝가리 유람선 침몰 3명 사망 16명 실종 ""대부분 아시아계""",국제>아시아,국제>유럽_EU,국제>중남미,NaN,NaN,NaN,NaN,"아시아,헝가리,부다페스트,다뉴브강,동아시아",NaN,"사망,헝가리,유람선,침몰,16명,실종,아시아계,다뉴브강,헝가리,부다페스트,34명,하...","헝가리,유람선,탑승객,3명,여행객,부다페스트,아시아,로이터,하블라니,아시아계,헝가리...",저녁 헝가리 부다페스트 다뉴브강에서 34명이 탄 '하블라니'(헝가리어로 '인어') ...,http://www.segye.com/content/html/2019/05/30/2...,NaN
1038,1.100401e+06,20190530,동아일보,김소정,"“헝가리 유람선 침몰, 7명 사망 19명 실종 한국인 33명 탑승”",국제>아시아,국제>유럽_EU,NaN,NaN,NaN,NaN,NaN,"헝가리,부다페스트,한국,부다강,다뉴브강","외교부,주헝가리대사관,재외국민보호대책본부","침몰,헝가리,유람선,사망,19명,실종,한국인,탑승,헝가리,수도,부다페스트,다뉴브강,...","한국인,헝가리,영사조력,유람선,7명,부다페스트,외교부,여행객,사망자,구조작업,다뉴브...",헝가리 수도 부다페스트 다뉴브강에서 29일(현지시간) 침몰한 유람선에 다수의 한국인...,http://news.donga.com/3/all/20190530/95763613/2,NaN
1039,1.100901e+06,20190530,중앙일보,이지영(lee.jiyoung2@joongang.co.kr),외교부 “헝가리 침몰 유람선에 한국인 33명 탑승 7명 사망 19명 실종”,국제>아시아,국제>유럽_EU,NaN,NaN,NaN,NaN,NaN,"헝가리,부다페스트,한국,다뉴브강","외교부,주헝가리대사관,재외국민보호대책본부","외교,헝가리,침몰,유람선,탑승,한국인,7명,사망,19명,실종,다뉴브강,헝가리,부다페...","헝가리,한국인,영사조력,7명,유람선,외교부,헝가리인,부다페스트,승무원,구조작업,실종...",헝가리 부다페스트 다뉴브강에서 29일(현지시간) 한국인 단체여행객 33명과 헝가리인...,https://news.joins.com/article/olink/23075866,NaN
1040,1.100901e+06,20190530,중앙일보,심새롬(saerom@joongang.co.kr),"다뉴브강의 비극 유람선 침몰, 한국인 7명 사망 19명 실종",국제>아시아,국제>유럽_EU,국제>중남미,사고>산업사고>화재,NaN,NaN,문재인,"헝가리,부다페스트,한국,다뉴브강","헝가리,국회,정부,국회의사당,외교부,다뉴브강","다뉴브강,비극,유람선,침몰,실종,한국인,사망,구조자,사망,7명,생존,난항,나머지,실...","헝가리,유람선,한국인,외교부,관광객,구조자,7명,실종자,다뉴브강,부다페스트,한국,피...",한국인 단체 관광객 33명이 탑승한 유람선이 헝가리 부다페스트에서 침몰했다. 현재까...,https://news.joins.com/article/olink/23075857,NaN


In [188]:
tokenlist={}

with open("tokenlist.txt",'r',encoding='utf-8') as f:
    for line in f:
        token,word=line.split(',')
        tokenlist[token]=word.strip()

front=[]
end=[]

for key in tokenlist.keys():
    first,second=key.split('_')
    front.append(first)
    end.append(second)

docs=[]

for article in df['키워드']:


    arr=article.split(',')
    
    del_nums=[]
    new_arr=[]
    for i in range(len(arr)-1):
        if arr[i] in front:
            if arr[i+1]==end[front.index(arr[i])]:
                print(arr[i],arr[i+1])
                del_nums.append(i)
                del_nums.append(i+1)
                #arr.append(arr[i]+" "+arr[i+1])
                arr.append(tokenlist[arr[i]+"_"+arr[i+1]])

      
    for i in range(len(arr)):
        if(i not in list(set(del_nums))):
             new_arr.append(arr[i])   
        
    #print(new_arr)
    docs.append(new_arr)


#docs
print("Finished")
    
        # for key in tokenlist.keys():
        #     front,end=key.split('_')

        #     for i in range(len(arr)-1):
        #         if(arr[i]==front):
        #             if(arr[i+1]==end):
        #                 #print(arr[i],arr[i+1])
        #                 new_arr.append(arr[i]+arr[i+1])
        #                 i=i+2
        #         else:
        #             new_arr.append(arr[i])

       



정상 회담
정상 회담
정상 회담
대북 제재
패스트 트랙
증거 인멸
증거 인멸
최저 임금
증거 인멸
도널드 트럼프
북미 정상
정상 회담
특별 대표
정상 회담
북미 정상
정상 회담
북미 정상
탄력 근로
최저 임금
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
패스트 트랙
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
비례 대표
비례 대표
증거 인멸
증거 인멸
정상 회담
도널드 트럼프
도널드 트럼프
트럼프 대통령
관세 부과
트럼프 대통령
관세 부과
트럼프 대통령
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
구속 영장
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
증거 인멸
구속 영장
증거 인멸
증거 인멸
구속 영장
구속 영장
실질 심사
구속 영장
실질 심사
구속 영장
부장 판사
도널드 트럼프
북미 정상
정상 회담
정상 회담
트럼프 대통령
정상 회담
정상 회담
특별 대표
북미 정상
정상 회담
정상 회담
북미 정상
정상 회담
도널드 트럼프
구속 영장
증거 인멸
북미 정상
정상 회담
북미 정상
정상 회담
특별 대표
영장 청구
구속 영장
특별 대표
피의자 신분
피의자 신분
피의자 신분
상응 조치
Finished


In [189]:
df2=df
df2['N_list']=docs
df2.to_csv("./Data/헝가리 유람선 사고_token.csv", encoding="utf-8-sig", index=False)

In [50]:
from collections import Counter

df1 = pd.read_csv("./Data/북미 정상 회담_token.csv", encoding='utf-8-sig')
df2 = pd.read_csv("./Data/강원도 산불_token.csv", encoding='utf-8-sig')
df3 = pd.read_csv("./Data/남북미 정상 회담_token.csv", encoding='utf-8-sig')
df4 = pd.read_csv("./Data/헝가리 유람선 사고_token.csv", encoding='utf-8-sig')


dfs=[]
dfs.append(df1)
dfs.append(df2)
dfs.append(df3)
dfs.append(df4)

whole=[]

words_cluster=pd.DataFrame()

for item in dfs:
    words=[]
    for line in item['N_list']:
        arr=re.sub("[\[,'\]]", '', line).split(' ')
        words+=arr    
    count=Counter(words).most_common(100)
    word = [key for key, item in count]
    whole.append(word)


print(whole)
words_cluster['topics']=whole
words_cluster.to_csv("./Data/2019_cluster_topics2.csv", encoding="utf-8-sig", index=False)

[['북한', '정상회담', '미국', '위원장', '대통령', '도널드트럼프', '회담', '김정은', '북미', '북미정상', '협상', '하노이', '한반도', '대화', '정상', '비핵화', '평화', '중국', '남북', '합의', '베트남', '정부', '국무', '한국', '이날', '트럼프', '방문', '한미', '가능성', '외교', '장관', '강조', '입장', '제재', '논의', '상황', '경제', '생각', '국무위원장', '재개', '문재인', '대북', '폼페이오', '청와대', '결렬', '대표', '가능', '주석', '진행', '언급', '일본', '백악관', '미사일', '조치', '진전', '요구', '국가', '러시아', '비건', '대북제재', '일정', '워싱턴', '의지', '완전', '평가', '보도', '영변', '회의', '폐기', '전망', '현지시간', '양국', '설명', '결과', '역할', '개최', '예정', '유지', '부위원장', '중요', '노력', '평양', '협력', '준비', '발언', '안보', '친서', '해제', '해결', '단계', '추진', '의원', '국무장관', '실무협상', '특별대표', '의미', '싱가포르', '제재완화', '볼턴', '방안'], ['산불', '대통령', '강원도', '피해', '지역', '정부', '발생', '지원', '국회', '재난', '대표', '고성', '강원', '한국당', '대응', '청와대', '국민', '상황', '이날', '화재', '장관', '현장', '의원', '속초', '회의', '문재인', '진화', '추경', '안전', '국가', '이재민', '복구', '원내', '주민', '대책', '한전', '원내대표', '방송', '생각', '예산', '시작', '비판', '강릉', '고성군', '바람', '전국', '소방관', '강조', '북한', '대피', '경제', '논의', '자유', '지적', '총리', '설명', '소방', 

In [234]:
whole_whole=[]


In [291]:
from collections import Counter

df1 = pd.read_csv("./Data/북미 정상 회담_token.csv", encoding='utf-8-sig')
df2 = pd.read_csv("./Data/강원도 산불_token.csv", encoding='utf-8-sig')
df3 = pd.read_csv("./Data/남북미 정상 회담_token.csv", encoding='utf-8-sig')
df4 = pd.read_csv("./Data/헝가리 유람선 사고_token.csv", encoding='utf-8-sig')

df1=df1.sample(n=int(len(df1)*0.3))
df2=df2.sample(n=int(len(df2)*0.3))
df3=df3.sample(n=int(len(df3)*0.3))
df4=df4.sample(n=int(len(df4)*0.3))



dfs=[]
dfs.append(df1)
dfs.append(df2)
dfs.append(df3)
dfs.append(df4)

whole=[]

for item in dfs:
    words=[]
    for line in item['N_list']:
        arr=re.sub("[\[,'\]]", '', line).split(' ')
        words+=arr    
    count=Counter(words).most_common(1)
    word = [key for key, item in count]
    whole.append(word)


print(whole)

#words_cluster.to_csv("./Data/2019_cluster_topics.csv", encoding="utf-8-sig", index=False)

[['북한', '정상회담', '미국', '위원장', '도널드트럼프', '대통령', '회담', '북미', '김정은', '협상'], ['산불', '대통령', '피해', '강원도', '정부', '지역', '발생', '지원', '국회', '재난'], ['대통령', '정상회담', '북한', '도널드트럼프', '위원장', '미국', '정상', '회담', '대화', '한반도'], ['헝가리', '사고', '유람선', '침몰', '다뉴브강', '한국인', '부다페스트', '수색', '현지', '구조']]


In [285]:
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



def get_topics(percent): 

    df1 = pd.read_csv("./Data/북미 정상 회담_token.csv", encoding='utf-8-sig')
    df2 = pd.read_csv("./Data/강원도 산불_token.csv", encoding='utf-8-sig')
    df3 = pd.read_csv("./Data/남북미 정상 회담_token.csv", encoding='utf-8-sig')
    df4 = pd.read_csv("./Data/헝가리 유람선 사고_token.csv", encoding='utf-8-sig')

    df1=df1.sample(n=int(len(df1)*percent))
    df2=df2.sample(n=int(len(df2)*percent))
    df3=df3.sample(n=int(len(df3)*percent))
    df4=df4.sample(n=int(len(df4)*percent))

    dfs=[]
    dfs.append(df1)
    dfs.append(df2)
    dfs.append(df3)
    dfs.append(df4)

    whole=[]

    for item in dfs:
        words=[]
        for line in item['N_list']:
            arr=re.sub("[\[,'\]]", '', line).split(' ')
            words+=arr    
        count=Counter(words).most_common(100)
        word = [key for key, item in count]
        whole.append(word)


    return whole
#print(whole)





#코사인 유사도 계산
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

sims=[]
for i in range(100):
    doc_nouns_list=[]

    cluster_topic = get_topics(1)[1]
    oasis_topic= get_topics(0.1)[1]

    doc_nouns_list.append(' '.join(cluster_topic))
    doc_nouns_list.append(' '.join(oasis_topic))

    #print("KNC_topic : ",doc_nouns_list[0])
    #print("Oasis_topic : ",doc_nouns_list[1])

    tfidf_vect_simple = TfidfVectorizer(min_df=1)
    feature_vect_simple = tfidf_vect_simple.fit_transform(doc_nouns_list)

    # TFidfVectorizer로 transform()한 결과는 Sparse Matrix이므로 Dense Matrix로 변환. 
    feature_vect_dense = feature_vect_simple.todense()

    #첫번째 문장과 두번째 문장의 feature vector  추출
    vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
    vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

    #첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
    similarity_simple = cos_similarity(vect1, vect2)

    print('KNC_topic, Oasis_topic Cosine 유사도: {0:.3f}'.format(similarity_simple))
    sims.append(similarity_simple)

sims=np.array(sims)
sims.mean()

KNC_topic, Oasis_topic Cosine 유사도: 0.629
KNC_topic, Oasis_topic Cosine 유사도: 0.542
KNC_topic, Oasis_topic Cosine 유사도: 0.616
KNC_topic, Oasis_topic Cosine 유사도: 0.566
KNC_topic, Oasis_topic Cosine 유사도: 0.616
KNC_topic, Oasis_topic Cosine 유사도: 0.603
KNC_topic, Oasis_topic Cosine 유사도: 0.518
KNC_topic, Oasis_topic Cosine 유사도: 0.578
KNC_topic, Oasis_topic Cosine 유사도: 0.629
KNC_topic, Oasis_topic Cosine 유사도: 0.518
KNC_topic, Oasis_topic Cosine 유사도: 0.616
KNC_topic, Oasis_topic Cosine 유사도: 0.603


KeyboardInterrupt: 

In [64]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

doc_nouns_list=[]

#코사인 유사도 계산
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

    

cluster_topic = ['도널드트럼프', '북한', '정상회담', '미국', '김정은', '핵', '중국', '협상', '회담', '북미', '차', '대통령', '정상', '하노이', '대화', '문재인', '미', '간', '무역', '가능']
bigkinds_topic=['북한', '대통령', '정상회담', '도널드트럼프', '위원장', '미국', '대화', '한반도', '회담', '평화', '정상', '김정은', '남북', '협상', '북미', '비핵화', '남북미', '한국', '북미정상', '하노이']

doc_nouns_list.append(' '.join(cluster_topic))
doc_nouns_list.append(' '.join(bigkinds_topic))

print("KNC_topic : ",doc_nouns_list[0])
print("Oasis_topic : ",doc_nouns_list[1])

tfidf_vect_simple = TfidfVectorizer(min_df=1,analyzer = 'char')
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_nouns_list)

# TFidfVectorizer로 transform()한 결과는 Sparse Matrix이므로 Dense Matrix로 변환. 
feature_vect_dense = feature_vect_simple.todense()

#첫번째 문장과 두번째 문장의 feature vector  추출
vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

#첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
similarity_simple = cos_similarity(vect1, vect2)

print('KNC_topic, Oasis_topic Cosine 유사도: {0:.3f}'.format(similarity_simple))

KNC_topic :  도널드트럼프 북한 정상회담 미국 김정은 핵 중국 협상 회담 북미 차 대통령 정상 하노이 대화 문재인 미 간 무역 가능
Oasis_topic :  북한 대통령 정상회담 도널드트럼프 위원장 미국 대화 한반도 회담 평화 정상 김정은 남북 협상 북미 비핵화 남북미 한국 북미정상 하노이
KNC_topic, Oasis_topic Cosine 유사도: 0.938


In [70]:
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



def get_topics(percent): 

    df1 = pd.read_csv("./Data/북미 정상 회담_TFIDF.csv", encoding='utf-8-sig')


    df1=df1.sample(n=int(len(df1)*percent))
    dfs=[]
    dfs.append(df1)


    whole=[]

    for item in dfs:
        words=[]
        for line in item['topics']:
            arr=re.sub("[\[,'\]]", '', line).split(' ')
            words+=arr    
        count=Counter(words).most_common(15)
        word = [key for key, item in count]
        whole.append(word)


    return whole





#코사인 유사도 계산
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

sims=[]

for i in range(10):
    doc_nouns_list=[]

    cluster_topic = get_topics(1)[0]
    oasis_topic= get_topics(0.7)[0]
    print('---------------')
    print(cluster_topic)
    print(oasis_topic)
    doc_nouns_list.append(' '.join(cluster_topic))
    doc_nouns_list.append(' '.join(oasis_topic))

    #print("KNC_topic : ",doc_nouns_list[0])
    #print("Oasis_topic : ",doc_nouns_list[1])

    tfidf_vect_simple = TfidfVectorizer(min_df=1,analyzer = 'char')
    feature_vect_simple = tfidf_vect_simple.fit_transform(doc_nouns_list)

    # TFidfVectorizer로 transform()한 결과는 Sparse Matrix이므로 Dense Matrix로 변환. 
    feature_vect_dense = feature_vect_simple.todense()

    #첫번째 문장과 두번째 문장의 feature vector  추출
    vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
    vect2 = np.array(feature_vect_dense[1]).reshape(-1,)

    #첫번째 문장과 두번째 문장의 feature vector로 두개 문장의 Cosine 유사도 추출
    similarity_simple = cos_similarity(vect1, vect2)

    print('KNC_topic, Oasis_topic Cosine 유사도: {0:.3f}'.format(similarity_simple))
    sims.append(similarity_simple)

sims=np.array(sims)
sims.mean()

---------------
['북한', '도널드트럼프', '위원장', '대통령', '정상회담', '중국', '미국', '대화', '베트남', '한미', '장관', '대표', '남북', '주석', '한반도']
['북한', '대통령', '도널드트럼프', '위원장', '정상회담', '중국', '대화', '미국', '한미', '베트남', '대표', '장관', '남북', '주석', '의원']
KNC_topic, Oasis_topic Cosine 유사도: 0.987
---------------
['북한', '도널드트럼프', '위원장', '대통령', '정상회담', '중국', '미국', '대화', '베트남', '한미', '장관', '대표', '남북', '주석', '한반도']
['북한', '위원장', '대통령', '도널드트럼프', '정상회담', '중국', '대화', '베트남', '미국', '대표', '장관', '남북', '한미', '주석', '의원']
KNC_topic, Oasis_topic Cosine 유사도: 0.987
---------------
['북한', '도널드트럼프', '위원장', '대통령', '정상회담', '중국', '미국', '대화', '베트남', '한미', '장관', '남북', '대표', '주석', '한반도']
['북한', '대통령', '위원장', '도널드트럼프', '정상회담', '미국', '중국', '베트남', '대화', '한미', '장관', '남북', '대표', '주석', '러시아']
KNC_topic, Oasis_topic Cosine 유사도: 0.981
---------------
['북한', '도널드트럼프', '위원장', '대통령', '정상회담', '중국', '미국', '대화', '베트남', '한미', '장관', '남북', '대표', '주석', '한반도']
['북한', '대통령', '도널드트럼프', '위원장', '정상회담', '중국', '미국', '대화', '베트남', '한미', '장관', '남북', '대표', '러시아', '의원']
KNC_top

0.9794197938046733